In [0]:
import pandas as pd
import numpy as np

# Load the training set
data=pd.read_csv('/content/drive/My Drive/tag_identification_dataset.csv',encoding='unicode_escape')
train_text = data['question']
train_labels = data['tag']

# Load the test set
df= pd.read_csv('/content/drive/My Drive/Test_case.csv',encoding='unicode_escape')
test_text = df['question']

In [0]:
# Combining questions from both training and test set 
text=[]
text=train_text.append(test_text)

In [0]:
# Encoding tags
from sklearn.preprocessing import LabelBinarizer
encoder1 = LabelBinarizer()
encoder1.fit(train_labels)
y_data = encoder1.transform(train_labels)

In [0]:
# Tokenization 
from keras.preprocessing.text import Tokenizer
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(text)
length = len(word_tokenizer.word_index) + 1
embedded_question = word_tokenizer.texts_to_sequences(text)

In [0]:
# Padding 
from nltk.tokenize import word_tokenize
from keras.preprocessing.sequence import pad_sequences
count = lambda question: len(word_tokenize(question))
longest_question = max(text, key=count)
longest_question = len(word_tokenize(longest_question))+5 
padded_question = pad_sequences(embedded_question, longest_question, padding='post')

In [0]:
# Create layers
from keras.models import Sequential
from keras.layers import Dense, Activation,Flatten
from keras.layers.embeddings import Embedding
model = Sequential()
model.add(Embedding(length, 20, input_length=longest_question))
model.add(Flatten())
model.add(Dense(7, activation='softmax'))

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
# Break the training and test set
ques=pd.DataFrame(padded_question)
train=ques.iloc[:107,].values
test=ques.iloc[107:,].values

In [0]:
# Fit the model
model.fit(train, y_data, epochs=200, verbose=1)

In [0]:
# Predict the tags
predictions = model.predict(test)
tag_labels = encoder1.classes_
predicted_tags = tag_labels[np.argmax(predictions,axis=1)]
print (predicted_tags)


In [0]:
# Convert into csv file
tag=predicted_tags.tolist()
df['tag']=tag
df.to_csv('akshaymor8287.csv')